# Generate sample for FID evaluation

In [ ]:
import torch, json, tqdm, os, shutil, numpy as np

from omegaconf import OmegaConf
from modules.util import instantiate_from_config
from models.diffusion.ddim import DDIMSampler
from models.diffusion.plms import PLMSSampler

from PIL import Image
from einops import rearrange
from torchvision.utils import make_grid
import torch

ROOT_PATH = "/mnt/data1/bardella_data/gitRepos/Thesis/ldm_porting"

dataset_dir = "/mnt/data1/bardella_data/gitRepos/Thesis/Datasets/wikiart"
dataset_subdivided_dir = "/mnt/data1/bardella_data/gitRepos/Thesis/Datasets/wikiart_subdivided_256"
mapping_file = "/mnt/data1/bardella_data/gitRepos/Thesis/dataset_preprocessing/wikiart/mapping.json"


# Load the label literal-discrete mapping
with open(mapping_file, "r") as fin:
    mapping = json.load(fin)
    mapping_r = {v:k for k, v in mapping}

# Load the dataset image path and labels
with open(dataset_dir+"/dataset.json", "r") as fin:
    dataset_info = json.load(fin)["labels"]

## Dataset per color subdivision and metadata generation

In [ ]:
# # Dataset Subdivision per color label
# new_dataset_info = {"labels":[]}

# if not os.path.exists(dataset_subdivided_dir):
#     for data in tqdm.tqdm(dataset_info):
#         img_dir, label = data

#         img_name, mapped_label = img_dir.split("/")[1], mapping_r[label]

#         dest_dir = "/".join((dataset_subdivided_dir,f"{mapped_label}"))

#         new_dataset_info["labels"].append(["/".join((mapped_label, img_name)), label])

#         if not os.path.exists(dest_dir):
#             os.makedirs(dest_dir)
#         shutil.copy(dataset_dir + f"/{img_dir}", dest_dir)
        

#     with open(dataset_subdivided_dir+"/dataset.json", "w") as fout:
#         json.dump(new_dataset_info, fout)

In [ ]:
from tqdm import tqdm
import cv2
from albumentations.augmentations.geometric.resize import Resize

if not os.path.exists(dataset_subdivided_dir):
    new_dataset_info = {"labels":[]}

    src_dir = dataset_dir
    dest_dir = dataset_subdivided_dir
    resize_transform = Resize(height=256, width=256,always_apply=True)

    for image_path, numerical_label in tqdm(dataset_info[:None]):

        image_name, letteral_label = image_path.split("/")[-1], mapping_r[numerical_label]

        image = cv2.imread(src_dir + f"/{image_path}")
        image_cvt = resize_transform(image=image)["image"]

        new_dataset_info["labels"].append(["/".join((letteral_label, image_name)), numerical_label])

        if not os.path.exists(dest_dir + f"/{letteral_label}"):
            os.makedirs(dest_dir + f"/{letteral_label}")
        
        cv2.imwrite(dest_dir + f"/{letteral_label}/{image_name}", image_cvt)

    with open(dest_dir+"/dataset.json", "w") as fout:
        json.dump(new_dataset_info, fout)
else:
    print("This subdivision already exist")

## Generate New Samples from the LD model

Load the pretrained LDM model

In [ ]:
def load_model_from_config(config, ckpt):
    print(f"Loading model from {ckpt}")
    pl_sd = torch.load(ckpt, map_location="cpu")
    sd = pl_sd["state_dict"]
    model = instantiate_from_config(config.model)
    m, u = model.load_state_dict(sd, strict=False)
    device = torch.device("cuda:1")
    model.to(device)
    model.eval()

    return model


config = OmegaConf.load(ROOT_PATH + "/configs/custom-ldm-cwa-vq-f8.yaml")  
sample_folder = ROOT_PATH + f"/sample/ldm/wikiart"

vq_gan_pretrained_ckpt_path = ROOT_PATH + "/pretrained_model/vq-f8/model.ckpt"
ldm_pretrained_ckpt_path = ROOT_PATH + "/model_checkpts/ldm/wikiart/epoch=299-step=69600.ckpt"
config.model.params.first_stage_config.params["ckpt_path"] = vq_gan_pretrained_ckpt_path

model = load_model_from_config(config, ldm_pretrained_ckpt_path)
# sampler = DDIMSampler(model)
# sampler_type = "DDIM"
sampler = PLMSSampler(model)
sampler_type = "PLMS"

Generate new Sample for a specific class. 
This operation is very time consuming and require a lot of memory.

The problem is we can only take about 15 samples at a time and it takes a few minuts for each batch.

To sample 10_000 with 200 denoising timesteps we need 11 hours

Pytorch implementation

In [2]:
import subprocess
import numpy

def computeFidPytorch(truth_dataset, generated_dataset, device):

    output = subprocess.run(['python', 
                            '-m', 
                            'pytorch_fid', 
                            truth_dataset, 
                            generated_dataset, 
                            "--device", 
                            device], stdout=subprocess.PIPE).stdout.decode('utf-8')
    

    return numpy.float32(output.split(" ")[-1][:-1])

In [ ]:
# classes = [0,1,2,3,4,5,6,7,8]   # define classes to be sampled here
classes = [1, 2]   # Choose the classes that we want the model sample from

# As said by the official TF FID repo we should generate at least 10000 sample 
# in order to have a FID which is not sub-optimal 

n_samples_per_class = 10_000 

max_batch_size = 15
FIDs = {}
if os.path.exists(ROOT_PATH + "/FIDs.npy"):
    with open(ROOT_PATH + "/FIDs.npy", "rb") as fin:
        FIDs = np.load(fin, allow_pickle=True).item()

whenFID = [1000 * i for i in range(1, 11)]

unconditional_class = 8

# ddim_steps = [20, 50, 200, 500, 1000]
ddim_steps = [75]
ddim_eta = 0
scale = 2  # for unconditional guidance

for ddim_step in ddim_steps:
    for cls in classes:

        # Create the folder where to save the sample
        sample_already_created = 0
        save_folder = sample_folder + f"/{sampler_type}/{ddim_step}/{mapping_r[cls]}"
        if not os.path.exists(save_folder):
            os.makedirs(save_folder)
        else:
            sample_already_created = len([file for file in os.listdir(save_folder) if os.path.isfile(os.path.join(save_folder, file))])

        image_counter = sample_already_created
        with torch.no_grad():
            with model.ema_scope():
                
                n_samples_per_class -= sample_already_created
                print(f"\nRendering {n_samples_per_class} examples of class '{mapping_r[cls]}' in {ddim_step} steps and using s={scale:.2f}.")
                if n_samples_per_class > max_batch_size:
                    sub_batches = [max_batch_size] * (n_samples_per_class//max_batch_size)
                    
                    if n_samples_per_class % max_batch_size != 0 :
                        sub_batches.append(n_samples_per_class % max_batch_size)

                else:
                    sub_batches = [n_samples_per_class]

                for idx_sub_batch, sub_batch_size in enumerate(sub_batches):
                    # Un-conditional class for free classifier sampling
                    uc = model.get_learned_conditioning({model.cond_stage_key: torch.tensor(sub_batch_size*[unconditional_class]).to(model.device)}) if scale != 1 else None
                    
                    #Conditional class
                    xc = torch.tensor(sub_batch_size*[cls])
                    c = model.get_learned_conditioning({model.cond_stage_key: xc.to(model.device)})
                    
                    samples_ddim, _ = sampler.sample(S=ddim_step,
                                                    conditioning=c,
                                                    batch_size=sub_batch_size,
                                                    shape=[4, 32, 32],
                                                    verbose=False,
                                                    unconditional_guidance_scale=scale,
                                                    unconditional_conditioning=uc, 
                                                    eta=ddim_eta)

                    x_samples_ddim = model.decode_first_stage(samples_ddim)
                    x_samples_ddim = torch.clamp((x_samples_ddim+1.0)/2.0, min=0.0, max=1.0).cpu()

                    # Save the just generated images and delete
                    for idx_img, images in enumerate(torch.chunk(x_samples_ddim, chunks = sub_batch_size, dim = 0)):
                        
                        final_image = 255. * rearrange(torch.squeeze(images, dim=0), 'c h w -> h w c').numpy()
                        final_image = Image.fromarray(final_image.astype(np.uint8))
                        image_number = sample_already_created + idx_img + idx_sub_batch * sub_batch_size
                        final_image.save(save_folder+ f"/image_{image_number}.png")
                    
                        image_counter += 1

                        if image_counter in whenFID:
                            FIDs[f"{sampler_type}_{image_counter}_{mapping_r[cls]}"] = computeFidPytorch(
                                                                                        truth_dataset=dataset_subdivided_dir + f"/{mapping_r[cls]}", 
                                                                                        generated_dataset=save_folder, 
                                                                                        device="cuda:2"
                                                                                        )
                            with open("FIDs.npy", "wb") as fout:
                                np.save(fout, FIDs, allow_pickle=True)
                        

                    # Delete the sampling tensor to reuse GPU memory istantly
                    del x_samples_ddim
                    

Generated samples against the all dataset label by label

In [9]:
import shutil, os, tqdm

dest_dir = "/mnt/data1/bardella_data/gitRepos/Thesis/Datasets/wikiart_against"

labels = ["arancione", "bianco", "blu", "nero", "verde"]
gpu_number = 0
FIDs = {}
with open("FIDs/fids_label_vs_allDataset.txt", "w") as fout:
    
    # Make the folder with only the ground truth images for a specific label
    if os.path.exists(dest_dir):
        shutil.rmtree(dest_dir)
        os.makedirs(dest_dir)

    src_dir = "/mnt/data1/bardella_data/gitRepos/Thesis/Datasets/wikiart_subdivided_256"
    for label in labels:
        for file in tqdm.tqdm(os.listdir(src_dir + f"/{label}")):
                shutil.copy(src_dir + f"/{label}/{file}", dest_dir+f"/{label}_{file}")

    for label in labels:
        print(label)

        FIDs[label] = computeFidPytorch(truth_dataset=dest_dir,
                                    generated_dataset=f"/mnt/data1/bardella_data/gitRepos/Thesis/ldm_porting/sample/ldm/wikiart/PLMS/75/{label}",
                                    device=f"cuda:{gpu_number}",)
        fout.write(f"{label} vs all dataset: {FIDs[label]}\n")

100%|██████████| 2493/2493 [00:00<00:00, 6338.12it/s]


arancione


100%|██████████| 400/400 [02:28<00:00,  2.69it/s]


bianco


100%|██████████| 200/200 [01:03<00:00,  3.17it/s]


...

100%|██████████| 200/200 [00:58<00:00,  3.45it/s]


nero


100%|██████████| 200/200 [00:40<00:00,  4.98it/s]


verde


100%|██████████| 200/200 [00:25<00:00,  7.99it/s]


Generated samples against only its corresponding label

In [6]:
import shutil, os, tqdm

dest_dir = "/mnt/data1/bardella_data/gitRepos/Thesis/Datasets/wikiart_against"

labels = ["arancione", "bianco", "blu", "nero", "verde"]
gpu_number = 0
FIDs = {}
with open("/FIDs/fids_label_vs_label.txt", "w") as fout:
    for label in labels:
        print(label)

        # Make the folder with only the ground truth images for a specific label
        if os.path.exists(dest_dir):
            shutil.rmtree(dest_dir)
            os.makedirs(dest_dir)
            

        src_dir = "/mnt/data1/bardella_data/gitRepos/Thesis/Datasets/wikiart_subdivided_256"

        
        for file in tqdm.tqdm(os.listdir(src_dir + f"/{label}")):
            shutil.copy(src_dir + f"/{label}/{file}", dest_dir+f"/{label}_{file}")


        FIDs[label] = computeFidPytorch(truth_dataset=dest_dir,
                                    generated_dataset=f"/mnt/data1/bardella_data/gitRepos/Thesis/ldm_porting/sample/ldm/wikiart/PLMS/75/{label}",
                                    device=f"cuda:{gpu_number}",)
        fout.write(f"{label} vs {label}: {FIDs[label]}\n")



arancione


100%|██████████| 400/400 [02:12<00:00,  3.02it/s]


bianco


100%|██████████| 200/200 [01:05<00:00,  3.05it/s]


blu


100%|██████████| 200/200 [00:58<00:00,  3.41it/s]


nero


100%|██████████| 200/200 [00:59<00:00,  3.35it/s]


verde


100%|██████████| 200/200 [00:55<00:00,  3.62it/s]


In [12]:
import numpy as np, os

FIDs = {}
if os.path.exists("/mnt/data1/bardella_data/gitRepos/Thesis/ldm_porting/FIDs/FIDs.npy"):
    with open("/mnt/data1/bardella_data/gitRepos/Thesis/ldm_porting/FIDs/FIDs.npy", "rb") as fin:
        FIDs = np.load(fin, allow_pickle=True).item()

import pprint

pprint.pprint(FIDs,indent=4)

{   'PLMS_10000_arancione': 100.54613,
    'PLMS_10000_blu': 158.08682,
    'PLMS_10000_verde': 102.91808,
    'PLMS_1000_arancione': 126.48265,
    'PLMS_1000_bianco': 110.8994,
    'PLMS_1000_blu': 165.49686,
    'PLMS_1000_verde': 126.81854,
    'PLMS_11000_arancione': 100.53096,
    'PLMS_12000_arancione': 100.344,
    'PLMS_13000_arancione': 100.192444,
    'PLMS_14000_arancione': 100.260376,
    'PLMS_15000_arancione': 100.21661,
    'PLMS_16000_arancione': 99.946884,
    'PLMS_17000_arancione': 99.967636,
    'PLMS_18000_arancione': 99.93763,
    'PLMS_19000_arancione': 99.84171,
    'PLMS_20000_arancione': 99.76372,
    'PLMS_2000_arancione': 111.11384,
    'PLMS_2000_blu': 162.07695,
    'PLMS_2000_verde': 113.297745,
    'PLMS_3000_arancione': 106.053406,
    'PLMS_3000_blu': 160.27895,
    'PLMS_3000_verde': 107.59354,
    'PLMS_4000_arancione': 104.049576,
    'PLMS_4000_blu': 159.28232,
    'PLMS_4000_verde': 106.30329,
    'PLMS_5000_arancione': 102.7775,
    'PLMS_5000_b

In [ ]:
import shutil, tqdm, os
labels = ["arancione","nero", "bianco"]
src_dir = "/mnt/data1/bardella_data/gitRepos/Thesis/Datasets/wikiart_subdivided_256"
dest_dir = "/mnt/data1/bardella_data/gitRepos/Thesis/Datasets/wikiart_against"
for label in labels:
    for file in tqdm.tqdm(os.listdir(src_dir + f"/{label}")):
        shutil.copy(src_dir + f"/{label}/{file}", dest_dir+f"/{label}_{file}")

In [ ]:
import shutil, tqdm, os
labels = ["arancione","nero", "bianco"]
src_dir = "/mnt/data1/bardella_data/gitRepos/Thesis/ldm_porting/sample/ldm/wikiart/PLMS/75"
dest_dir = "/mnt/data1/bardella_data/gitRepos/Thesis/Datasets/wikiart_generated_256"
for label in labels:
    for file in tqdm.tqdm(os.listdir(src_dir + f"/{label}")[:9990]):
        shutil.copy(src_dir + f"/{label}/{file}", dest_dir+f"/{label}_{file}")

## Stylegan generation

In [ ]:
!git clone https://github.com/NVlabs/stylegan3.git "/mnt/data1/bardella_data/gitRepos/Thesis/ldm_porting/styleGan_repos/sg3"
!git clone https://github.com/NVlabs/stylegan2-ada-pytorch.git "/mnt/data1/bardella_data/gitRepos/Thesis/ldm_porting/styleGan_repos/sg2"

In [ ]:
import subprocess, os, json

checkPts_sg2_path = "/mnt/data1/bardella_data/gitRepos/Thesis/ldm_porting/pretrained_model/StyleGans/absfig-sg2.pkl"
stylegan2outFolder = "/mnt/data1/bardella_data/gitRepos/Thesis/ldm_porting/sample/styleGan2Out"
sg2_script_path = "/mnt/data1/bardella_data/gitRepos/Thesis/ldm_porting/styleGan_repos/sg2/generate.py"

checkPts_sg3_path = "/mnt/data1/bardella_data/gitRepos/Thesis/ldm_porting/pretrained_model/StyleGans/stylegan3-absfig-snapshot-000660.pkl"
stylegan3outFolder = "/mnt/data1/bardella_data/gitRepos/Thesis/ldm_porting/sample/styleGan3Out"
sg3_script_path = "/mnt/data1/bardella_data/gitRepos/Thesis/ldm_porting/styleGan_repos/sg3/gen_images.py"

mapping_file = "/mnt/data1/bardella_data/gitRepos/Thesis/dataset_preprocessing/wikiart/mapping.json"

n_classes = 8
n_samples = 10000

with open(mapping_file, "r") as fin:
    mapping = json.load(fin)
    mapping_r = {v:k for k, v in mapping}

In [ ]:
for outFolder, checkPts, script_path in zip([stylegan2outFolder, stylegan3outFolder], [checkPts_sg2_path, checkPts_sg3_path], [sg2_script_path, sg3_script_path]):

  for cls in range(n_classes):
    outFolderfinal = outFolder + f"/{mapping_r[cls]}"
    if not os.path.exists(outFolderfinal): 
      os.makedirs(outFolderfinal)

    subprocess.run(["python", script_path,
                      f"--outdir={outFolderfinal}",
                      f"--trunc={1}",
                      f"--seeds=0-{n_samples}",
                      f"--class={cls}",
                      f"--network={checkPts}",
                      ], 
                    )

# FID COMPUTATION

## TensorFlow inplementation

To run this cell we need to have tensorflow Version 1.1 installed with python 3.7

CLI command:

conda install -c anaconda tensorflow-gpu=1.15 python=3.7

In [ ]:
# import subprocess
# from pprint import pprint
# import numpy

# labels = ["arancione", "bianco", "blu", "giallo", "nero", "rosso", "verde", "viola"]
# truth_label = "arancione"
# generated_dataset = f"/mnt/data1/bardella_data/gitRepos/Thesis/ldm_porting/sample/ldm/wikiart/200/{truth_label}"

# device =  "0"

# FIDs = {}
# for label in labels:

#     truth_dataset = f"/mnt/data1/bardella_data/gitRepos/Thesis/Datasets/wikiart_subdivided/{label}"

#     output = subprocess.run(['/home/bardella/.conda/envs/thesis_tf/bin/python', 
#                             "/mnt/data1/bardella_data/gitRepos/Thesis/ldm_porting/modules/fid.py", 
#                             truth_dataset,
#                             generated_dataset,
#                             "--gpu",
#                             "0",
#                             ], stdout=subprocess.PIPE).stdout.decode('utf-8')

#     FIDs["_Vs_".join((label, truth_label))] = numpy.float32(output.split(" ")[-1][:-1])

# pprint(FIDs)

In [ ]:
# # Paths
# import modules.fid as fid
# import glob
# from skimage import imread
# import tensorflow as tf

# image_path = '/tmp/images' # set path to some generated images
# stats_path = 'fid_stats.npz' # training set statistics
# inception_path = fid.check_or_download_inception(None) # download inception network

# # loads all images into memory (this might require a lot of RAM!)
# image_list = glob.glob(os.path.join(image_path, '*.jpg'))
# images = np.array([imread(str(fn)).astype(np.float32) for fn in files])

# # load precalculated training set statistics
# f = np.load(stats_path)
# mu_real, sigma_real = f['mu'][:], f['sigma'][:]
# f.close()

# fid.create_inception_graph(inception_path)  # load the graph into the current TF graph
# with tf.Session() as sess:
#     sess.run(tf.global_variables_initializer())
#     mu_gen, sigma_gen = fid.calculate_activation_statistics(images, sess, batch_size=100)

# fid_value = fid.calculate_frechet_distance(mu_gen, sigma_gen, mu_real, sigma_real)
# print("FID: %s" % fid_value)